## Tranfer learning модели [Disaster-Twitter-XLM-RoBERTa-AL](https://huggingface.co/hannybal/disaster-twitter-xlm-roberta-al) в качестве фильтра новостей об опасных природных явлениях (ОПЯ) от других публикуемых.
##### На примере `РИА-Новости` и пабликов `Вконтакте`

In [2]:
import pandas as pd
import numpy as np
import time
import re
from collections import Counter
from transformers import pipeline

### Загружаем новости с ОПЯ

In [26]:
data = pd.read_excel('files//data_edited.xlsx')

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30053 entries, 0 to 30052
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   _ID                     30053 non-null  object        
 1   Event_ID                23866 non-null  object        
 2   News_link               30053 non-null  object        
 3   Text                    30053 non-null  object        
 4   News_length             30053 non-null  int64         
 5   Repeat_num              30053 non-null  int64         
 6   News_source             30053 non-null  object        
 7   News_type               30053 non-null  object        
 8   Location_ATD4           29914 non-null  object        
 9   Location_other          26845 non-null  object        
 10  Location_geocoded       30053 non-null  object        
 11  Location_geocoded_bool  30053 non-null  bool          
 12  Phenomena               30053 non-null  object

### Загрузка [модели](https://huggingface.co/hannybal/disaster-twitter-xlm-roberta-al), пример ее применяния и выводы.

In [41]:
ex = data['Text'].sample(random_state=666).values[0]
ex

'‼ ШТОРМОВОЕ ПРЕДУПРЕЖДЕНИЕ ‼  По прогнозу Тульского центра по гидрометеорологии и мониторингу окружающей среды в ближайшие 1-3 часа с сохранением вечером 17 мая 2018 по территории Тульской области ожидаются грозы с усилением юго-восточного ветра до 12-17 м/с.    Главное управление МЧС России по Тульской области напоминает о соблюдении мер предосторожности и правил поведения усилении ветра:    📢 Рекомендации для населения при сильном ветре:    1. Машину поставьте в гараж, при отсутствии гаража машину следует парковать вдали от деревьев, а также слабо укрепленных конструкций.  2. Находясь на улице, обходите рекламные щиты, шаткие строения и дома с неустойчивой кровлей.  3. Избегайте деревьев и разнообразных сооружений повышенного риска (мостов, эстакад, трубопроводов, линий электропередач, потенциально опасных промышленных объектов).    Главное управление МЧС России по Тульской области предупреждает, что поджоги сухой травы на дачах и приусадебных участках могут привести к пожарам, как 

In [42]:
MODEL_NAME = 'hannybal/disaster-twitter-xlm-roberta-al'
classifier = pipeline('text-classification', model=MODEL_NAME, tokenizer='cardiffnlp/twitter-xlm-roberta-base')
classifier(ex)

Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.999849796295166}]

### Тестироание модели на БД ОПЯ, считается, что все новости затрагивают тематику ОПЯ

In [ ]:
start_time = time.time()

def to_def_disaster(text):
    # Ограничиваем длину текста
    max_length = 512
    if len(text) > max_length:
        text = text[:max_length]
    dictt = classifier(text)[0]
    if dictt['label'] == 'LABEL_1':
        return 'ОПЯ', round(dictt['score'] * 100, 3)
    else:
        return 'не ОПЯ', round(dictt['score'] * 100, 3)


results = data['Text'].apply(to_def_disaster)

In [15]:
data[['result', 'conf']] = pd.DataFrame(results.tolist(), index=data.index)

end_time = time.time()
execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time//60:.2f} минут")


Время выполнения: 194.00 минут


In [30]:
# data.to_excel('files//data_edited.xlsx', index = False)

In [18]:
data['result'].value_counts()

result
ОПЯ       26570
не ОПЯ     3486
Name: count, dtype: int64

In [21]:
data[data['result']=='ОПЯ']['conf'].describe()

count    26570.000000
mean        99.635523
std          2.799763
min         50.097000
25%         99.971000
50%         99.987000
75%         99.991000
max         99.992000
Name: conf, dtype: float64

In [28]:
for i in data[data['result']=='не ОПЯ']['Text'].sample(5):
    print(i)
    print()

⚠️ ВНИМАНИЕ ⚠️ ШТОРМОВОЕ ПРЕДУПРЕЖДЕНИЕ об опасных природных явлениях 10 декабря в республике ожидается понижение температуры воздуха ночью местами до -40,-45, туман, гололедица. Служба ГО и ЧС Республики Тыва жителей республики призывает во избежание переохлаждения и обморожения без необходимости не выходить на улицу, избегать длительного пребывания на улице детей и домашних животных и воздержаться от дальних поездок. ☎️ Телефон экстренных служб 112

По данным Астраханского ЦГМС днем 27-28.06.2018 в период с 13 час. 00 мин. до 17 час. 00 мин. по Астраханской области ожидается сильная жара 40°. В связи с этим МЧС рекомендует: При сильной жаре следует избегать воздействия повышенной температуры. Носите светлую воздухопроницаемую одежду (желательно из хлопка) с головным убором. Находясь под прямым воздействием солнечных лучей, закрывать как можно больше поверхности кожи. Помните, что обожженная кожа перестает выделять пот и охлаждаться. Передвигайтесь не спеша, старайтесь чаще находиться

### Загружаем новости с `РИА-Новости` за период с 15 марта 2018 года по 31 декабря 2018 года. Их можно скачать с [диска](https://drive.google.com/file/d/12MyiSFXbN7ULaneFM3sB6aRgofJFXVAG/view?usp=sharing)

In [ ]:
import gdown

url = "https://drive.google.com/drive/folders/11oCcLplWtp_qm-WuEbfCFP_Mz5K_z3ps?usp=sharing"
gdown.download_folder(url, quiet=True, use_cookies=False)

df_ria = pd.read_csv('news_data/ria_news.tsv', sep='\t')
df_ria = df_ria[~df_ria.tags.isnull()]
print(df_ria.shape)
df_ria.head()

In [4]:
# data = pd.read_excel('files//data_edited.xlsx')

In [5]:
df_ria = pd.read_csv('files//Новости//ria_news.tsv', sep='\t')
df_ria = df_ria[~df_ria.tags.isnull()]
print(df_ria.shape)
df_ria.head()

(201708, 9)


,href,date,time,title,snippet,text,category,tags,shows
0,/20181231/1548961410.html,2018-12-31,"31 декабря 2018, 23:52",Нетаньяху не собирается в отставку в случае пр...,Премьер-министр Израиля Биньямин Нетаньяху не ...,"МОСКВА, 31 дек - РИА Новости. Премьер-министр ...",В мире,"Биньямин Нетаньяху, Израиль, В мире",728.0
1,/20181231/1548961364.html,2018-12-31,"31 декабря 2018, 23:19",Макрон в новогоднем обращении затронул тему ре...,"Результат реформ не может быть мгновенным, зая...","ПАРИЖ, 31 дек – РИА Новости. Результат реформ ...",В мире,"Эммануэль Макрон, Франция, В мире",3086.0
2,/20181231/1548961337.html,2018-12-31,"31 декабря 2018, 23:12",Аарон Рэмзи проведет переговоры с пятью топ-кл...,"Полузащитник лондонского ""Арсенала"" Аарон Рэмз...","МОСКВА, 31 дек - РИА Новости. Полузащитник лон...",NaN,ФК Арсенал (Лондон),183.0
3,/20181231/1548961304.html,2018-12-31,"31 декабря 2018, 23:09",Гол Азмуна принес сборной Ирана победу над кат...,Футболисты сборной Ирана одержали победу над к...,"МОСКВА, 31 дек - РИА Новости. Футболисты сборн...",NaN,"Сердар Азмун, Сборная Ирана по футболу",78.0
4,/20181231/1548961265.html,2018-12-31,"31 декабря 2018, 23:07",Пятая ракетка мира дель Потро пропустит Открыт...,Аргентинский теннисист Хуан Мартин дель Потро ...,"МОСКВА, 31 дек - РИА Новости. Аргентинский тен...",NaN,Теннис,79.0


In [6]:
# удалим все лишние пробелы и сделаем lowercase
df_ria['tags'] = (
    df_ria.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(df_ria.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))
# tags_cnt.most_common()[-20:]
target_tags = {tag for tag,cnt in tags_cnt.most_common() if cnt > 10}
len(target_tags)

13344


3039

In [7]:
df_ria['category'].value_counts()

category
В мире                             50073
Спорт                              31630
Происшествия                       18279
Общество                           16957
Экономика                          13738
Политика                            7959
Недвижимость                        7306
Наука                               4655
Культура                            4151
Туризм                              3391
Сказано в эфире - Радио Sputnik     3196
Религия                             2684
Социальный навигатор                2442
Акценты - Радио Sputnik             1045
Видеоблоги - Радио Sputnik           461
Навигатор абитуриента                353
Программы - Радио Sputnik            191
Радио                                171
Инфографика                          155
Теория заблуждений                    40
АртВывих                              20
Спецпроекты - Радио Sputnik            2
Name: count, dtype: int64

### Рассмотрим категорию `спорт`. Ее самые распространенные тематики

In [66]:
sports = df_ria[df_ria['category']=='Спорт'].dropna(subset=['text']).sample(20000)

In [68]:
sports['tags'] = (
    sports.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(sports.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))

1211


In [70]:
top_10 = sorted(tags_cnt.items(), key=lambda item: item[1], reverse=True)[:10]

for key, value in top_10:
    print(f'{key}: {value}')

спорт: 20062
футбол: 11188
хоккей: 3611
чемпионат мира по футболу 2018: 2747
новости - чемпионат мира по футболу 2018: 2017
теннис: 1536
единоборства: 1359
баскетбол: 1054
фигурное катание: 660
сборная россии - чемпионат мира по футболу 2018: 603


### Так как в этой рубрике точно нет новостей об ОПЯ, поэтому можно использовать ее для оценки качества работы этой [модели](https://huggingface.co/hannybal/disaster-twitter-xlm-roberta-al) в качестве фильтра информативных новостей от других публикуемых.

Приеним модель к новостям спорта

In [71]:
start_time = time.time()

def to_def_disaster(text):
    # Ограничиваем длину текста
    max_length = 512
    if len(text) > max_length:
        text = text[:max_length]
    dictt = classifier(text)[0]
    if dictt['label'] == 'LABEL_1':
        return 'ОПЯ', round(dictt['score'] * 100, 3)
    else:
        return 'не ОПЯ', round(dictt['score'] * 100, 3)



results = sports['text'].apply(to_def_disaster)
sports[['result', 'conf']] = pd.DataFrame(results.tolist(), index=sports.index)

end_time = time.time()
execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time//60:.2f} минут")


Время выполнения: 176.00 минут


In [72]:
# sports.to_excel('files//sports.xlsx', index = False)

In [73]:
sports['result'].value_counts()

result
не ОПЯ    19895
ОПЯ         105
Name: count, dtype: int64

In [3]:
# sports = pd.read_excel('files//sports.xlsx')

Возьмем проклассифицированные новости из БД ОПЯ, источник которых соц. сеть `Вхонтакте`.

После отсмотра случайных 1% пабликов `ВКонтакте` (было условие включить разные регионы, чтобы никакие 3 сообщества не был из одного региона), из которых брались новости для БД ОПЯ было установлено, что новости об ОПЯ составляют 4.43 % всех новостей. 

Возьмем: 
- 4.43 % - новости `Вконтакте` (927)
- 95.57 % - новости спорта `РИА-Новости` (20000)

Случайно выберем из базы 927 новостей `Вконтакте` и посчитаем метрики качества классификации

In [22]:
sample_data = data[data['News_source']=='ВКонтакте'].sample(927, random_state = 666)

In [23]:
sample_data['result'].value_counts()

result
ОПЯ       812
не ОПЯ    115
Name: count, dtype: int64

In [24]:
sports['result'].value_counts()

result
не ОПЯ    19895
ОПЯ         105
Name: count, dtype: int64

In [25]:
TP = len(sample_data[sample_data['result']=='ОПЯ']) # модель угадала что ОПЯ - ОПЯ
FN = len(sample_data[sample_data['result']=='не ОПЯ']) # модель сказала, что это не ОПЯ, но это ОПЯ
FP = len(sports[sports['result']=='ОПЯ']) # Модель сказала, что это ОПЯ, но это не ОПЯ
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
print(f'Precision: {Precision:.2f}')
print(f'Recall: {Recall:.2f}')

Precision: 0.89
Recall: 0.88


## Рассмотрим несколько рубрик на предмет содержания новостей об ОПЯ
#### Для создания БД ОПЯ из СМИ (`РИА-Новости` в т.ч.) брались только новости из рубрики `Проишествия`, но новости об ОПЯ, есть не только там.
Рубрики:
- Ощество
- Недвижимость
- Проишествия

### Рассмотрим категорию `Общество`. Ее самые распространенные тематики.

In [8]:
society = df_ria[df_ria['category']=='Общество']

In [35]:
society['tags'] = (
    society.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(society.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))

2279


In [36]:
top_10 = sorted(tags_cnt.items(), key=lambda item: item[1], reverse=True)[:10]

for key, value in top_10:
    print(f'{key}: {value}')

общество: 16867
россия: 7224
москва: 3259
владимир путин: 1003
госдума рф: 650
крым: 450
пожар в торговом центре в кемерово: 347
кемерово: 302
экология: 295
московская область: 276


### Посмотрим, сколько в этой рубрике потенциальных новостей об ОПЯ.

In [11]:
start_time = time.time()

def to_def_disaster(text):
    # Ограничиваем длину текста
    max_length = 512
    if len(text) > max_length:
        text = text[:max_length]
    dictt = classifier(text)[0]
    if dictt['label'] == 'LABEL_1':
        return 'ОПЯ', round(dictt['score'] * 100, 3)
    else:
        return 'не ОПЯ', round(dictt['score'] * 100, 3)


# Удаляем строки, где в столбце 'text' есть NaN
society = society.dropna(subset=['text'])
results = society['text'].apply(to_def_disaster)
society[['result', 'conf']] = pd.DataFrame(results.tolist(), index=society.index)

end_time = time.time()
execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time//60:.2f} минут")


Время выполнения: 104.00 минут


In [12]:
# society.to_excel('files//society.xlsx')

In [13]:
society['result'].value_counts()

result
не ОПЯ    13330
ОПЯ        3537
Name: count, dtype: int64

Получается, 3537 новостей из категории `Общество` скорее всего содержат в себе информацию об ОПЯ

### Посмотрим на самые популярные тематики в новостях категории `Общество`, которые были определены моделью как новости, содержащи ОПЯ.

In [33]:
society_hazards = society[society['result']=='ОПЯ']

# удалим все лишние пробелы и сделаем lowercase
society_hazards['tags'] = (
    society_hazards.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(society_hazards.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))

742


C:\Users\Mariia\AppData\Local\Temp\ipykernel_7920\2049203375.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  society_hazards['tags'] = (


In [34]:
top_10 = sorted(tags_cnt.items(), key=lambda item: item[1], reverse=True)[:10]

for key, value in top_10:
    print(f'{key}: {value}')

общество: 3537
москва: 1311
россия: 490
пожар в торговом центре в кемерово: 308
кемерово: 262
крым: 171
московская область: 153
нападение на керченский политехнический колледж: 130
керчь: 118
экология: 97


In [32]:
for i in society_hazards[society_hazards['tags'].str.contains(r'\b(ураган)\b', regex=True)]['text'].sample(5):
    print(i)
    print()


МОСКВА, 22 апр — РИА Новости. Порывы ветра в московском регионе в субботу во время урагана достигали 30 метров в секунду, в столице с ливнем выпало 6 миллиметров осадков, а в Подмосковье — почти половина от месячной нормы, сообщил РИА Новости специалист центра погоды "Фобос" Евгений Тишковец.
 "В Москве порывы 16 метров в секунду, в Домодедово — 22 метра в секунду, Волоколамск — 23, Шереметьево — 24, во Внуково порывы достигали 30 метров в секунду", — сказал Тишковец. 
"Если у нас месячная норма 44 миллиметра осадков. То получатся, что там почти пятьдесят процентов осадков выпало", — уточнил Тишковец.
Сильнейший ураган и ливни прошлись по Москве днем в субботу. По последним данным, в столице из-за урагана пострадали 17 человек, 14 из них были госпитализированы, среди них трое детей, погибла 12-летняя девочка. Сообщалось также, что в Подмосковье один человек погиб, шестеро пострадали.
 

БРЯНСК, 21 апр – РИА Новости. Энергоснабжение 440 населенных пунктов в Ивановской, Тверской, Владим

C:\Users\Mariia\AppData\Local\Temp\ipykernel_7920\2618698173.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  for i in society_hazards[society_hazards['tags'].str.contains(r'\b(ураган)\b', regex=True)]['text'].sample(5):


### Попробуем рассмотреть категорию `Недвижимость`, так как этот раздел весьма схож с тем, что публикуют ВК паблики. Посмотрим, сколько в этой рубрике потенциальных новостей об ОПЯ.

In [60]:
import time

start_time = time.time()

def to_def_disaster(text):
    # Ограничиваем длину текста
    max_length = 512
    if len(text) > max_length:
        text = text[:max_length]
    dictt = classifier(text)[0]
    if dictt['label'] == 'LABEL_1':
        return 'ОПЯ', round(dictt['score'] * 100, 3)
    else:
        return 'не ОПЯ', round(dictt['score'] * 100, 3)


# Удаляем строки, где в столбце 'text' есть NaN
house = house.dropna(subset=['text'])
results = house['text'].apply(to_def_disaster)
house[['result', 'conf']] = pd.DataFrame(results.tolist(), index=house.index)

end_time = time.time()
execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time//60:.2f} минут")


Время выполнения: 55.00 минут


In [61]:
house['result'].value_counts()

result
не ОПЯ    5289
ОПЯ       1787
Name: count, dtype: int64

In [8]:
# house.to_excel('files//Новости//house.xlsx', index = False)

### Посмотрим на самые популярные тематики в новостях категории `Недвижимость`, которые были определены моделью как новости, содержащие ОПЯ.

In [12]:
house_hazards = house[house['result']=='ОПЯ']

# удалим все лишние пробелы и сделаем lowercase
house_hazards['tags'] = (
    house_hazards.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(house_hazards.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))

949


C:\Users\Mariia\AppData\Local\Temp\ipykernel_1948\2317000029.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_hazards['tags'] = (


In [13]:
top_10 = sorted(tags_cnt.items(), key=lambda item: item[1], reverse=True)[:10]

for key, value in top_10:
    print(f'{key}: {value}')

новости - недвижимость: 1762
россия: 1494
москва: 818
инфраструктура - новости: 506
строительство: 370
жилье - новости: 324
жилье: 245
инфраструктура: 198
московская область: 177
жкх: 169


In [16]:
for i in house_hazards[house_hazards['tags'].str.contains(r'\b(ураган|наводнение)\b', regex=True)]['text'].sample(5):
    print(i)
    print()

 ПЯТИГОРСК, 21 ноя — РИА Новости. Суд в Ставропольском крае подтвердил правомерность выдачи жилищных сертификатов шести семьям, пострадавшим от паводка в 2017 году, сообщает министерство ЖКХ региона. В июне текущего года в ходе прямой линии с президентом была озвучена проблема нескольких семей, которые лишились жилья после паводка. Им несколько раз отказывали в выдаче жилищных сертификатов, так как дома либо уже подвергались подтоплению в 2002 году и на них ранее выдавались сертификаты, либо оформлены в долевую собственность. Девять семей обратились в суд. "Суд вынес решения уже по 6 делам. Все решения подтверждают правоту позиции правительства Ставропольского края о том, что данные заявители имеют право на получение государственного жилищного сертификата, и обращены к немедленному исполнению", — говорится в сообщении. Как отмечается, специалисты министерства ЖКХ края максимально оперативно подготовят документы 6 семей, по которым принято положительное решение суда, и повторно направят

C:\Users\Mariia\AppData\Local\Temp\ipykernel_1948\94866683.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  for i in house_hazards[house_hazards['tags'].str.contains(r'\b(ураган|наводнение)\b', regex=True)]['text'].sample(5):


### Рассмотрим категорию `Происшествия`. Посмотрим, сколько в этой рубрике потенциальных новостей об ОПЯ.

In [74]:
events = df_ria[df_ria['category']=='Происшествия']

In [75]:
start_time = time.time()

def to_def_disaster(text):
    # Ограничиваем длину текста
    max_length = 512
    if len(text) > max_length:
        text = text[:max_length]
    dictt = classifier(text)[0]
    if dictt['label'] == 'LABEL_1':
        return 'ОПЯ', round(dictt['score'] * 100, 3)
    else:
        return 'не ОПЯ', round(dictt['score'] * 100, 3)


# Удаляем строки, где в столбце 'text' есть NaN
events = events.dropna(subset=['text'])
results = events['text'].apply(to_def_disaster)
events[['result', 'conf']] = pd.DataFrame(results.tolist(), index=events.index)

end_time = time.time()
execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time//60:.2f} минут")


Время выполнения: 114.00 минут


In [77]:
events['result'].value_counts()

result
не ОПЯ    9610
ОПЯ       8661
Name: count, dtype: int64

In [19]:
# events.to_excel('files//Новости//events.xlsx', index = False)

### Посмотрим на самые популярные тематики в новостях категории `Происшествия`, которые были определены моделью как новости, содержащие ОПЯ

In [21]:
events_hazards = events[events['result']=='ОПЯ']

In [22]:
events_hazards_hazards = events_hazards[events_hazards['tags'].str.contains(r'\b(мчс|паводок|наводнение|ураган|' + \
                                             r'обрушение кровли здания|обрушение моста в приморье|' + \
                                             r'обрушение|экология|гумчс|аварии|экологическая|водоканал|' +  \
                                             r'гэс|природные пожары|авиалесоохрана|русгидро|пожарной безопасности|'+ \
                                             r'обрушение в здании)\b', regex=True)]
print(len(events_hazards_hazards))

484


C:\Users\Mariia\AppData\Local\Temp\ipykernel_1948\3200604211.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  events_hazards_hazards = events_hazards[events_hazards['tags'].str.contains(r'\b(мчс|паводок|наводнение|ураган|' + \


In [23]:
events_hazards = events[events['result']=='ОПЯ']

# удалим все лишние пробелы и сделаем lowercase
events_hazards['tags'] = (
    events_hazards.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(events_hazards.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))

968


C:\Users\Mariia\AppData\Local\Temp\ipykernel_1948\1297991010.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_hazards['tags'] = (


In [24]:
top_10 = sorted(tags_cnt.items(), key=lambda item: item[1], reverse=True)[:10]

for key, value in top_10:
    print(f'{key}: {value}')

происшествия: 8661
москва: 777
пожар в торговом центре в кемерово: 545
кемерово: 528
россия: 464
санкт-петербург: 278
московская область: 255
нападение на керченский политехнический колледж: 235
керчь: 235
краснодарский край: 174


In [25]:
for i in events_hazards[events_hazards['tags'].str.contains(r'\b(ураган|наводнение)\b', regex=True)]['text'].sample(5):
    print(i)
    print()

СОЧИ, 7 ноя – РИА Новости. Порядка 70% пострадавших при подтоплении на Кубани получили выплаты, общая сумма компенсаций составила более 373 миллионов рублей, сообщает пресс-служба администрации региона. По информации краевых властей, все выплаты пострадавшим за частичную и полную утрату имущества сделаны из краевого бюджета. "В результате паводка в трех районах Кубани подтопленными оказались 29 социальных объектов и 2798 домов с населением 8652 человека, из них 258 домовладений признаны аварийными. На сегодняшний день выплаты получили порядка 70% пострадавших в ЧС, для этого значительный объем средств был заимствован из ряда социально значимых краевых программ. Общая сумма компенсаций составила более 373 миллионов рублей", — сообщает пресс-служба со ссылкой на губернатора края Вениамина Кондратьева. Уточняется, что на сегодняшний день основные восстановительные работы завершены, транспортное сообщение, а также электро- и водоснабжение возобновлены во всех пострадавших муниципалитетах. 

C:\Users\Mariia\AppData\Local\Temp\ipykernel_1948\229537244.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  for i in events_hazards[events_hazards['tags'].str.contains(r'\b(ураган|наводнение)\b', regex=True)]['text'].sample(5):
